In [ ]:
import numpy as np
from time import time

The following function computes the log normal density as we assume that the stock price follows a lognormal distribution,

In [ ]:
def logNormal(S, r, q, sig, S0, T):
    # Computes the log normal density function for the stock price
    f = np.exp(-0.5*((np.log(S/S0)-(r-q-sig**2/2)*T)/(sig*np.sqrt(T)))**2) / (sig*S*np.sqrt(2*np.pi*T))
    return f

Note: when evaluating the function at  $S=0.0$ , the theoretical value should be  $f=0.0$ , but the function above will give you a warning and a NaN value. Instead this can be evaluated at  $S=ϵ$ , where  ϵ  is a small positive number such as, $1e-8$.

With the following parameters we can price a European put:

In [ ]:
# Parameters for put
S0 = 100
K = 90
r = 0.04
q = 0.02
sig = 0.25
T = 1.0

This first example demonstrates a numerical integration method to price the put. This function takes the above parameters as arguments and a positive integer  N  which is the number of grid points:

In [ ]:
def numerical_integral_put(r, q, S0, K, sig, T, N):
    ''' Numerical integration for puts. '''

    # spacing of the grid
    eta = 0.0
    # price of the put
    priceP = 0.0

    # discount factor
    df = np.exp(-r * T)
    # step size
    eta = 1. * K / N
    # vector of stock price
    S = np.arange(0, N) * eta
    # avoid numerical issues
    S[0] = 1e-8
    # vector of weights
    w = np.ones(N) * eta
    w[0] = eta / 2
    # lognormal densities
    logN = np.zeros(N)
    logN = logNormal(S, r, q, sig, S0, T)
    # numerical integral
    sumP = np.sum((K - S) * logN * w)
    # price put
    priceP = df * sumP

    return eta, priceP

Use the previous function to price the put for different values of $ N=2n,  n=1,2,…,15$

In [ ]:
# vector with all values of n
n_min = 1
n_max = 15
n_vec = np.arange(n_min, n_max + 1, dtype=int)

In [ ]:
# compute the numerical integration for each value of n
start_time = time()
# will store the results in vectors
eta_vec = np.zeros(n_max)
put_vec = np.zeros(n_max)
for i in range(n_max):
    N = 2** n_vec[i]

    [eta_vec[i], put_vec[i]] = numerical_integral_put(r, q, S0, K, sig, T, N)

end_time = time()
print('Elapsed time (seconds): ' + str(end_time - start_time))

Elapsed time (seconds): 0.008443355560302734


In [ ]:
# print a table with the numerical integration for each value of n
print('N\teta\tP_0')
for i in range(n_max):
    print('2^%i\t%.3f\t%.4f' % (n_vec[i], eta_vec[i], put_vec[i]))

N	eta	P_0
2^1	45.000	0.4847
2^2	22.500	3.8251
2^3	11.250	4.3546
2^4	5.625	4.4822
2^5	2.812	4.5137
2^6	1.406	4.5216
2^7	0.703	4.5236
2^8	0.352	4.5241
2^9	0.176	4.5242
2^10	0.088	4.5242
2^11	0.044	4.5242
2^12	0.022	4.5242
2^13	0.011	4.5242
2^14	0.005	4.5242
2^15	0.003	4.5242


## Black-Merton-Scholes (BMS) Model

A formula that assumes that an assets price follows Brownian motion but the asset's price volatility is constant,

$$ dS_{t} = (r-q)S_{t}dt + σS_{t}dW_{t} $$

Characteristic Function of the log of the stock price,

$$ \phi(ν) = e^{i \bigl(ln \ S_{0} + (r-q-{\frac{σ^{2}}{2}})T\bigl)ν-{\frac{σ^{2}ν^{2}}{2}T}} $$

In this example we assume,

$$ Θ = \begin{Bmatrix} σ \end{Bmatrix} =  \begin{Bmatrix} 3.0 \end{Bmatrix}  $$

## Heston Stochastic Volatility Model

The following formulas describe the movement of asset prices, where an asset's price and volatility follow random, Brownian motion processes.

The five parameters for the Heston Model:
* $κ$: rate of reversion to the long-term price variance
* $θ$: long term price variance
* $λ$: price of volatility risk
* $ρ$: the corelation coefficent between both brownian motions
* $\text{v}_{0}$: initial point where you start variance

$S_{t}$ follows the following SDE,

$$ dS_{t} = (r-q)S_{t}dt + \sqrt{\text{v}_{t}}S_{t} dW_{t}^{(1)}$$

$$ d\text{v}_{t} = κ(θ-\text{v}_{t})dt + λ\sqrt{\text{v}_{t}} dW_{t}^{(2)}$$

Characteristic Function of the log of the stock price,

$$ ϕ(u) = \frac{e^{ \begin{Bmatrix} {iu \space ln \space S_{0} + iu(r-q)T + \frac{κ \space θ \space T \space (κ - \space i \space ρ \space σ \space u)
}{σ^2}}\end{Bmatrix}}}{(\text{cosh} \space \frac{γ \space T}{2}+ \frac{κ-i \space ρ \space σ \space u}{γ} \space \text{sinh} \space \frac{γ \space T}{2})^{\frac{2 \space κ \space θ}{σ^2}} } \space * \space e^{\begin{Bmatrix} \frac{-(u^2+iu) \space v_{0}}{γ \space \text{coth}\space \frac{γ \space T}{2} + \space κ - \space i \space ρ \space σ \space u} \end{Bmatrix}} $$

Where,

$$ γ = \sqrt{σ^2(u^2+iu) + (κ-i \space ρ \space σ \space u )^2}$$

In this example we assume,

$$ Θ = \begin{Bmatrix} κ, θ, λ, ρ, \text{v}_{0} \end{Bmatrix} =  \begin{Bmatrix} 2, 0.05, 0.3, -0.7, 0.04 \end{Bmatrix}$$

## Variance Gamma (VG) Model

The VG process $X(t;σ, ν, θ)$ is obtained by evaluating a Brownian motion $W(t)$ with drift $ b(t; θ, \nu) = θt + σW(t) $ at a random time given by a gamma process with unit mean rate, $ γ \space (t:1, ν)$  

The three parameters for the VG process:
* $σ$: volatility of the brownian motion
* $\nu$: kurtosis, variance rate of the gamma time change
* $θ$: skewness, drift of the Brownian motion with drift

$ S_{t} $ is given by,

$$ S_{t} = S_{0} e^{(r-q+ ω)t + X(t;σ,ν,θ)} $$

Characteristic function of the log of the stock price,

$$ ϕ(ν) = e^{(iu(ln \space S_{0}+(r-q)T)} \Bigl (\frac{1}{1-iuθν+σ^{2}u^{2}ν/2} \Bigl )^{\frac{T}{ν}}  $$

In this example we assume,

$$ Θ = \begin{Bmatrix} σ,\nu, \theta \end{Bmatrix} =  \begin{Bmatrix} 0.3, 0.5, -0.4 \end{Bmatrix}$$

---

Below is a function that computes the characteristic function of the log-stock price for three different models: Black Merton-Scholes, Heston, and Variance Gamma, each one contains different sets of parameters.

In [ ]:
def generic_CF(u, params, S0, r, q, T, model):
    ''' Computes the characteristic function for different models (BMS, Heston, VG). '''

    if (model == 'BMS'):
        # unpack parameters
        sig = params[0]
        # characteristic function
        mu = np.log(S0) + (r-q-sig**2/2)*T
        a = sig*np.sqrt(T)
        phi = np.exp(1j*mu*u-(a*u)**2/2)

    elif(model == 'Heston'):
        # unpack parameters
        kappa  = params[0]
        theta  = params[1]
        sigma  = params[2]
        rho    = params[3]
        v0     = params[4]
        # characteristic function
        tmp = (kappa-1j*rho*sigma*u)
        g = np.sqrt((sigma**2)*(u**2+1j*u)+tmp**2)
        pow1 = 2*kappa*theta/(sigma**2)
        numer1 = (kappa*theta*T*tmp)/(sigma**2) + 1j*u*T*r + 1j*u*np.log(S0)
        log_denum1 = pow1 * np.log(np.cosh(g*T/2)+(tmp/g)*np.sinh(g*T/2))
        tmp2 = ((u*u+1j*u)*v0)/(g/np.tanh(g*T/2)+tmp)
        log_phi = numer1 - log_denum1 - tmp2
        phi = np.exp(log_phi)

    elif (model == 'VG'):
        # unpack parameters
        sigma  = params[0];
        nu     = params[1];
        theta  = params[2];
        # characteristic function
        if (nu == 0):
            mu = np.log(S0) + (r-q - theta -0.5*sigma**2)*T
            phi  = np.exp(1j*u*mu) * np.exp((1j*theta*u-0.5*sigma**2*u**2)*T)
        else:
            mu  = np.log(S0) + (r-q + np.log(1-theta*nu-0.5*sigma**2*nu)/nu)*T
            phi = np.exp(1j*u*mu)*((1-1j*nu*theta*u+0.5*nu*sigma**2*u**2)**(-T/nu))

    return phi

We now provide you with a function that uses FFT to price European option for any of the 3 models we have discussed. The same function works for both calls and puts. It return two vectors, one with the log-strikes and one with option prices.

In [ ]:
def genericFFT(params, S0, K, r, q, T, alpha, eta, n, model):
    ''' Option pricing using FFT (model-free). '''

    N = 2**n

    # step-size in log strike space
    lda = (2 * np.pi / N) / eta

    # choice of beta
    #beta = np.log(S0)-N*lda/2 # the log strike we want is in the middle of the array
    beta = np.log(K) # the log strike we want is the first element of the array

    # forming vector x and strikes km for m=1,...,N
    km = np.zeros(N)
    xX = np.zeros(N)

    # discount factor
    df = np.exp(-r*T)

    nuJ = np.arange(N) * eta
    psi_nuJ = generic_CF(nuJ - (alpha + 1) * 1j, params, S0, r, q, T, model) / ((alpha + 1j*nuJ)*(alpha+1+1j*nuJ))

    km = beta + lda * np.arange(N)
    w = eta * np.ones(N)
    w[0] = eta / 2
    xX = np.exp(-1j * beta * nuJ) * df * psi_nuJ * w

    yY = np.fft.fft(xX)
    cT_km = np.zeros(N)
    multiplier = np.exp(-alpha * km) / np.pi
    cT_km = multiplier * np.real(yY)

    return km, cT_km

Estimate a European put with the following parameters,

In [ ]:
# parameters
S0 = 100
K = 80
r = 0.05
q = 0.01
T = 1.0

For the 3 models, consider the same values of $\alpha$, $\eta$, and $n$,

In [ ]:
# parameters for fft
eta_vec = np.array([0.1, 0.25])
n_vec = np.array([6, 10])
alpha_vec = np.array([-1.01, -1.25, -1.5, -1.75, -2., -5.])
num_prices = len(eta_vec) * len(n_vec) * len(alpha_vec)

Pricing function that given a model and the above parameters will compute the put price,

In [ ]:
def price_all_puts(params, S0, K, r, q, T, model, alpha_vec, eta_vec, n_vec):
    num_prices = len(eta_vec) * len(n_vec) * len(alpha_vec)
    # output is a matrix, the columns correspond to eta, n, alpha, and put price
    put_matrix = np.zeros([num_prices, 4])
    i = 0
    for eta in eta_vec:
        for n in n_vec:
            for alpha in alpha_vec:
                # pricing via fft
                put = 0 # store here the put price

                k_vec, option_vec = genericFFT(params, S0, K, r, q, T, alpha, eta, n, model)
                put = option_vec[0] # only interested in one strike

                put_matrix[i] = np.array([eta, n, alpha, put])
                i += 1
    return put_matrix

#### BMS


In [ ]:
# model-specific parameters
mod = 'BMS'
sig = 0.3
params = [sig]

In [ ]:
start_time = time()
bms_matrix = price_all_puts(params, S0, K, r, q, T, mod, alpha_vec, eta_vec, n_vec)
end_time = time()
print('Elapsed time (seconds): ' + str(end_time - start_time))

Elapsed time (seconds): 0.006446123123168945


In [ ]:
# print results in table form
print('Model = ' + mod)
print('eta\tN\talpha\tput')
for i in range(num_prices):
    print('%.2f\t2^%i\t%.2f\t%.4f' % (bms_matrix[i,0], bms_matrix[i,1], bms_matrix[i,2], bms_matrix[i,3]))

Model = BMS
eta	N	alpha	put
0.10	2^6	-1.01	89.7431
0.10	2^6	-1.25	2.7396
0.10	2^6	-1.50	2.7569
0.10	2^6	-1.75	2.7701
0.10	2^6	-2.00	2.7789
0.10	2^6	-5.00	2.6727
0.10	2^10	-1.01	89.7316
0.10	2^10	-1.25	2.7080
0.10	2^10	-1.50	2.7080
0.10	2^10	-1.75	2.7080
0.10	2^10	-2.00	2.7080
0.10	2^10	-5.00	2.7080
0.25	2^6	-1.01	269.0367
0.25	2^6	-1.25	2.8504
0.25	2^6	-1.50	2.7083
0.25	2^6	-1.75	2.7080
0.25	2^6	-2.00	2.7080
0.25	2^6	-5.00	2.7080
0.25	2^10	-1.01	269.0367
0.25	2^10	-1.25	2.8504
0.25	2^10	-1.50	2.7083
0.25	2^10	-1.75	2.7080
0.25	2^10	-2.00	2.7080
0.25	2^10	-5.00	2.7080


#### Heston

In [ ]:
# model-specific parameters
mod = 'Heston'
kappa = 2.
theta = 0.05
lda = 0.3
rho = -0.7
v0 = 0.04
params = [kappa, theta, lda, rho, v0]

In [ ]:
start_time = time()
heston_matrix = price_all_puts(params, S0, K, r, q, T, mod, alpha_vec, eta_vec, n_vec)
end_time = time()
print('Elapsed time (seconds): ' + str(end_time - start_time))

Elapsed time (seconds): 0.009718894958496094


In [ ]:
# print results in table form
print('Model = ' + mod)
print('eta\tN\talpha\tput')
for i in range(num_prices):
    print('%.2f\t2^%i\t%.2f\t%.4f' % (heston_matrix[i,0], heston_matrix[i,1], heston_matrix[i,2], heston_matrix[i,3]))

Model = Heston
eta	N	alpha	put
0.10	2^6	-1.01	88.0744
0.10	2^6	-1.25	1.1102
0.10	2^6	-1.50	1.1666
0.10	2^6	-1.75	1.2167
0.10	2^6	-2.00	1.2605
0.10	2^6	-5.00	1.3979
0.10	2^10	-1.01	88.3648
0.10	2^10	-1.25	1.3412
0.10	2^10	-1.50	1.3412
0.10	2^10	-1.75	1.3412
0.10	2^10	-2.00	1.3412
0.10	2^10	-5.00	1.3412
0.25	2^6	-1.01	267.6738
0.25	2^6	-1.25	1.4873
0.25	2^6	-1.50	1.3450
0.25	2^6	-1.75	1.3445
0.25	2^6	-2.00	1.3442
0.25	2^6	-5.00	1.3415
0.25	2^10	-1.01	267.6698
0.25	2^10	-1.25	1.4835
0.25	2^10	-1.50	1.3414
0.25	2^10	-1.75	1.3412
0.25	2^10	-2.00	1.3412
0.25	2^10	-5.00	1.3412


#### Variance Gamma

In [ ]:
# model-specific parameters
mod = 'VG'
sigma = 0.3
nu = 0.5
theta = -0.4
params = [sigma, nu, theta]

In [ ]:
start_time = time()
vg_matrix = price_all_puts(params, S0, K, r, q, T, mod, alpha_vec, eta_vec, n_vec)
end_time = time()
print('Elapsed time (seconds): ' + str(end_time - start_time))

Elapsed time (seconds): 0.00450897216796875


In [ ]:
# print results in table form
print('Model = ' + mod)
print('eta\tN\talpha\tput')
for i in range(num_prices):
    print('%.2f\t2^%i\t%.2f\t%.4f' % (vg_matrix[i,0], vg_matrix[i,1], vg_matrix[i,2], vg_matrix[i,3]))

Model = VG
eta	N	alpha	put
0.10	2^6	-1.01	92.2103
0.10	2^6	-1.25	5.2047
0.10	2^6	-1.50	5.2230
0.10	2^6	-1.75	5.2401
0.10	2^6	-2.00	5.2556
0.10	2^6	-5.00	0.0030
0.10	2^10	-1.01	92.3373
0.10	2^10	-1.25	5.3137
0.10	2^10	-1.50	5.3137
0.10	2^10	-1.75	5.3137
0.10	2^10	-2.00	5.3137
0.10	2^10	-5.00	0.0000
0.25	2^6	-1.01	271.6399
0.25	2^6	-1.25	5.4539
0.25	2^6	-1.50	5.3121
0.25	2^6	-1.75	5.3122
0.25	2^6	-2.00	5.3124
0.25	2^6	-5.00	0.0019
0.25	2^10	-1.01	271.6423
0.25	2^10	-1.25	5.4560
0.25	2^10	-1.50	5.3139
0.25	2^10	-1.75	5.3137
0.25	2^10	-2.00	5.3137
0.25	2^10	-5.00	0.0020
